In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.tabular.all import *
from wwf.tab.export import *
from bnb import *

from classes import *
from utils import *

In [3]:
SEASON = '21'

## Download HKJC odds

In [4]:
path_raw = Path('raw_data')
path_data = Path('data')

path_output = Path('output')

In [5]:
session = requests.Session()
r = session.get('http://bet.hkjc.com')
cookies = r.cookies

In [6]:
odds_url = 'https://bet.hkjc.com/football/getJSON.aspx?jsontype=odds_chl.aspx'
response = session.post(
    odds_url,
    headers={'referer':'http://bet.hkjc.com'},
    cookies=cookies
)

In [7]:
with open(path_data/'json'/f'odds_chl-{datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.txt', 'w') as f:
    f.write(response.text)

In [8]:
matches_json = json.loads(response.text)[1]['matches']

In [9]:
# Example
# matches_json = json.loads(open(path_data/'json'/'odds_chl-2021-09-21-14-03-17.txt').read())[1]['matches']

In [10]:
matches = [Match(m) for m in matches_json]
odds = [m.export() for m in matches]

['20211029FRI1', '2021-Oct-30', '00:00:00', 'Russian Premier [RPL]', 'Zenit St. Petersburg', 'Dinamo Moscow', 'false', '13.5', '4.15', '1.18', 'false', '11.5', '2.65', '1.42', 'true', '10.5', '2.05', '1.68']
['20211029FRI2', '2021-Oct-30', '00:30:00', 'German Division 2 [GD2]', 'Heidenheim', 'Schalke 04', 'false', '11.5', '2.45', '1.48', 'true', '10.5', '2.00', '1.72', 'false', '14.5', '5.55', '1.10']
['20211029FRI3', '2021-Oct-30', '00:30:00', 'German Division 2 [GD2]', 'Darmstadt', 'Nurnberg', 'false', '10.5', '2.20', '1.59', 'false', '13.5', '4.90', '1.13', 'true', '9.5', '1.75', '1.95']
['20211029FRI9', '2021-Oct-30', '02:30:00', 'German Division 1 [GSL]', 'Hoffenheim', 'Hertha Berlin', 'true', '9.5', '1.80', '1.90', 'false', '10.5', '2.25', '1.57', 'false', '13.5', '4.90', '1.13']
['20211029FRI12', '2021-Oct-30', '02:45:00', 'Eng Championship [ED1]', 'QPR', 'Nottingham Forest', 'true', '10.5', '2.05', '1.68', 'false', '11.5', '2.65', '1.42', 'false', '13.5', '4.30', '1.17']
['2021

In [11]:
cols_match = ['MatchDay', 'Date', 'Time', 'LeagueJC', 'HomeTeamJC', 'AwayTeamJC']
cols_odds0 = ['MAINLINE_0', 'CHL_LINE_0', 'CHL_H_0', 'CHL_L_0']
cols_odds1 = ['MAINLINE_1', 'CHL_LINE_1', 'CHL_H_1', 'CHL_L_1']
cols_odds2 = ['MAINLINE_2', 'CHL_LINE_2', 'CHL_H_2', 'CHL_L_2']
cols_odds  = ['MAINLINE', 'CHL_LINE', 'CHL_H', 'CHL_L']
cols_pred  = ['prob_0', 'prob_1', 'prob_2', 'total_count']

cols = cols_match + cols_odds0 + cols_odds1 + cols_odds2

In [12]:
odds = pd.DataFrame(odds, columns=cols)
odds = odds.fillna(value=np.nan)

In [13]:
cols_odds_ = cols_odds0[1:]+cols_odds1[1:]+cols_odds2[1:]
odds[cols_odds_] = odds[cols_odds_].astype(float)

In [14]:
odds['MatchDay'] = odds['MatchDay'].str[8:]
odds['Date'] = pd.to_datetime(odds['Date'])
odds['Time'] = pd.to_datetime(odds['Time'], format='%H:%M:%S').dt.time

In [15]:
odds['DateTimeJC'] = pd.to_datetime(odds['Date'].dt.date.map(str) + '-' + odds['Time'].map(str))
odds['DateTimeJC'] = odds['DateTimeJC'].dt.tz_localize('Hongkong')
odds['DateTime'] = odds['DateTimeJC'].dt.tz_convert('GB')

In [16]:
map_league = pd.read_csv(path_data/'league.csv')
map_team = pd.read_csv(path_data/'team.csv')

In [17]:
# Map Div name
odds = odds.merge(map_league[['LeagueJC', 'Div']], 'inner', on='LeagueJC')

In [18]:
# Map Team name
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'HomeTeam'}), 'inner', 
                  left_on='HomeTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])
odds = odds.merge(map_team[['TeamNameJC', 'TeamName']].rename(columns={'TeamName':'AwayTeam'}), 'inner', 
                  left_on='AwayTeamJC', right_on='TeamNameJC').drop(columns=['TeamNameJC'])

## Download recent stats

In [20]:
# Download latest results in current season
!wget -q https://www.football-data.co.uk/mmz4281/{SEASON}{int(SEASON)+1}/data.zip -O raw_data/data.zip

# Unzip to folder
!unzip -q -o raw_data/data.zip -d raw_data/{SEASON}

In [19]:
usecols = ['Div', 'Date', 'HomeTeam', 'AwayTeam', 'HC', 'AC', 'FTHG', 'FTAG', 'HS', 'AS', 'HST', 'AST']
dtype = {'HC':'float', 'AC':'float'}
parse_dates = ['Date']

seasons = [SEASON]

dfs = []

for folder in sorted(path_raw.iterdir()):
    if folder.is_dir() and folder.name in seasons: 
        for file in sorted(folder.glob('*.csv')):
            try:
                df = pd.read_csv(file, usecols=usecols, dtype=dtype, parse_dates=parse_dates, dayfirst=True)
                df['Season'] = folder.name
                dfs.append(df)
            except:
                continue

In [20]:
df_season = pd.concat(dfs)
df_season = df_season.dropna()
df_season = df_season.sort_values(['Div', 'Date', 'HomeTeam']).reset_index(drop=True)

In [21]:
df_hist = pd.read_csv(path_data/'data.csv', dtype={'HC':'float', 'AC':'float'}, parse_dates=['Date'])
df_hist = df_hist.query(f'Season == {int(SEASON)-1}').reset_index(drop=True)

In [22]:
df_season = pd.concat([df_hist[df_season.columns], df_season])

In [23]:
# Make features on historical stats (Home and Away)
stats = ['FTHG', 'HS', 'HST', 'HC', 'FTAG', 'AS', 'AST', 'AC']
df_home, df_away = joinLastGamesStatsHomeAway(df_season, stats)

In [24]:
# Make features on historical stats (For and Against)
stats = [('FTHG', 'FTAG', 'FTG'), ('HS', 'AS', 'S'), ('HST', 'AST', 'ST'), ('HC', 'AC', 'C')]
df_for, df_against = joinLastGamesStatsForAgainst(df_season, stats)

In [25]:
df_home = df_home.sort_values(['HomeTeam', 'Date']).reset_index(drop=True)
df_away = df_away.sort_values(['AwayTeam', 'Date']).reset_index(drop=True)

df_home = df_home.groupby('HomeTeam')[df_home.columns[df_home.columns.str.contains('Avg')]].last().reset_index()
df_away = df_away.groupby('AwayTeam')[df_away.columns[df_away.columns.str.contains('Avg')]].last().reset_index()

In [26]:
odds = odds.merge(df_home, 'left', 'HomeTeam').merge(df_away, 'left', 'AwayTeam')

In [27]:
cols_home = df_for.columns[df_for.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_for.columns[df_for.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_for = df_for.groupby('Team')[df_for.columns[df_for.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_for.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_for.rename(columns=cols_away), 'left', 'AwayTeam')

In [28]:
cols_home = df_against.columns[df_against.columns.str.contains('Avg')]
cols_home = dict(zip(cols_home, 'Home'+cols_home))
cols_home.update({'Team':'HomeTeam'})

cols_away = df_against.columns[df_against.columns.str.contains('Avg')]
cols_away = dict(zip(cols_home, 'Away'+cols_away))
cols_away.update({'Team':'AwayTeam'})

df_against = df_against.groupby('Team')[df_against.columns[df_against.columns.str.contains('Avg')]].last().reset_index()
odds = odds.merge(df_against.rename(columns=cols_home), 'left', 'HomeTeam').merge(df_against.rename(columns=cols_away), 'left', 'AwayTeam')

In [29]:
add_datepart(odds, 'DateTime', prefix='', drop=False);

In [30]:
display_df(odds.head(5).T)

,0,1,2,3,4
MatchDay,FRI2,FRI3,SAT17,SAT18,SAT19
Date,2021-10-30 00:00:00,2021-10-30 00:00:00,2021-10-30 00:00:00,2021-10-30 00:00:00,2021-10-30 00:00:00
Time,00:30:00,00:30:00,19:30:00,19:30:00,19:30:00
LeagueJC,German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2],German Division 2 [GD2]
HomeTeamJC,Heidenheim,Darmstadt,Dresden,Hannover,Werder Bremen
AwayTeamJC,Schalke 04,Nurnberg,Sandhausen,Aue,St. Pauli
MAINLINE_0,false,false,false,true,false
CHL_LINE_0,11.5,10.5,13.5,9.5,13.5
CHL_H_0,2.45,2.2,5.1,1.9,4.4
CHL_L_0,1.48,1.59,1.12,1.8,1.16


## Load model

In [31]:
learn_bnb = load_learner('models/learn_bnb.pkl')

In [32]:
to = load_pandas('models/to.pkl')

In [33]:
def predict(self, row):
    "Predict on a Pandas Series"
    dl = self.dls.test_dl(row.to_frame().T)
    dl.dataset.conts = dl.dataset.conts.astype(np.float32)
    inp,preds,_ = self.get_preds(dl=dl, with_input=True, with_decoded=False)
    b = tuplify(inp)
    full_dec = self.dls.decode(b)
    return full_dec,preds[0]

learn_bnb.predict = MethodType(predict, learn_bnb)

In [34]:
to_tst = to.new(odds)
to_tst.process()
# to_tst.items.head()

In [35]:
tst_dl = learn_bnb.dls.valid.new(to_tst)
tst_dl.show(max_n=999)

,Div,HomeTeam,AwayTeam,Dayofweek,FTHGLast5Avg,HSLast5Avg,HSTLast5Avg,HCLast5Avg,FTAGLast5Avg,ASLast5Avg,ASTLast5Avg,ACLast5Avg,HomeFTGForLast5Avg,HomeSForLast5Avg,HomeSTForLast5Avg,HomeCForLast5Avg,AwayFTGForLast5Avg,AwaySForLast5Avg,AwaySTForLast5Avg,AwayCForLast5Avg,HomeFTGAgainstLast5Avg,HomeSAgainstLast5Avg,HomeSTAgainstLast5Avg,HomeCAgainstLast5Avg,AwayFTGAgainstLast5Avg,AwaySAgainstLast5Avg,AwaySTAgainstLast5Avg,AwayCAgainstLast5Avg,Year,Month,Week,Day,Dayofyear
0,D2,Heidenheim,Schalke 04,4,1.0,16.6,4.40,9.20,1.4,12.6,4.8,3.6,1.8,15.0,4.8,6.6,1.6,15.2,5.6,4.4,2.0,12.0,4.2,7.0,0.4,9.4,2.0,3.8,2021.0,10.0,43.0,29.0,302.0
1,D2,Darmstadt,Nurnberg,4,3.2,12.8,5.60,3.20,1.6,12.0,5.2,4.2,2.4,13.4,6.0,3.6,1.2,11.8,3.8,4.6,1.0,12.0,3.6,6.0,0.8,14.6,4.4,6.4,2021.0,10.0,43.0,29.0,302.0
2,D2,Dresden,Sandhausen,5,2.0,14.0,6.75,4.75,1.4,11.8,4.4,5.4,0.8,11.2,3.0,4.0,1.2,7.6,3.0,3.0,1.4,14.8,4.8,6.4,2.6,18.8,7.0,7.2,2021.0,10.0,43.0,30.0,303.0
3,D2,Hannover,Erzgebirge Aue,5,0.8,14.6,5.60,7.00,0.6,9.8,3.2,3.0,1.0,12.4,3.4,4.6,1.0,13.2,4.4,3.8,0.6,13.4,4.0,5.0,2.2,14.0,4.6,5.2,2021.0,10.0,43.0,30.0,303.0
4,D2,Werder Bremen,St Pauli,5,1.4,16.6,5.80,6.00,0.8,9.8,3.6,3.4,1.2,15.2,4.4,6.0,2.8,14.4,6.6,5.4,1.6,9.4,3.8,3.2,1.0,11.2,3.0,3.8,2021.0,10.0,43.0,30.0,303.0
5,D2,Hamburg,Holstein Kiel,5,2.2,18.0,7.00,8.80,1.6,13.0,3.8,6.0,1.6,15.2,5.6,9.4,1.0,12.4,4.0,6.2,1.0,10.6,3.0,3.6,1.8,14.6,4.4,3.8,2021.0,10.0,43.0,30.0,303.0
6,D2,Karlsruhe,Paderborn,6,1.8,15.2,4.80,5.00,2.4,14.0,5.6,4.6,1.8,14.6,4.6,5.8,1.8,15.8,5.6,4.4,1.8,13.8,4.8,2.6,1.4,18.0,7.2,5.0,2021.0,10.0,43.0,31.0,304.0
7,D2,Ingolstadt,Regensburg,6,0.5,13.5,3.00,4.75,1.2,9.8,3.6,4.4,0.6,11.0,1.8,3.4,1.8,14.0,5.2,3.2,2.6,13.6,5.2,5.4,1.6,14.6,5.2,4.2,2021.0,10.0,43.0,31.0,304.0
8,D2,Hansa Rostock,Fortuna Dusseldorf,6,1.0,12.5,4.25,6.00,1.2,15.8,4.2,6.8,1.0,13.2,3.6,4.8,1.4,15.0,5.0,5.4,1.0,10.4,4.6,4.8,1.2,15.4,4.6,5.2,2021.0,10.0,43.0,31.0,304.0
9,D1,Hoffenheim,Hertha,4,2.2,14.2,4.60,5.60,1.0,7.6,4.0,2.8,1.8,13.2,3.6,4.4,1.6,9.8,3.8,4.0,1.2,11.0,2.8,4.8,2.2,13.8,5.6,5.8,2021.0,10.0,43.0,29.0,302.0


In [36]:
pred, _ = learn_bnb.get_preds(dl=tst_dl)

In [37]:
probs = F.softmax(pred[:, :3], dim=-1)
total_count = 1 / F.softplus(pred[:, 3:])

In [38]:
odds[cols_pred] = torch.cat([probs, total_count], dim=-1)

In [39]:
odds0 = odds[cols_match+cols_odds0+cols_pred].rename(columns=dict(zip(cols_odds0, cols_odds)))
odds1 = odds[cols_match+cols_odds1+cols_pred].rename(columns=dict(zip(cols_odds1, cols_odds)))
odds2 = odds[cols_match+cols_odds2+cols_pred].rename(columns=dict(zip(cols_odds2, cols_odds)))

In [40]:
odds = pd.concat([odds0, odds1, odds2]).dropna().reset_index(drop=True)
odds['MAINLINE'] = np.where(odds['MAINLINE']=='true', True, False)

In [41]:
odds.head(10)

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_0,prob_1,prob_2,total_count
0,FRI2,2021-10-30,00:30:00,German Division 2 [GD2],Heidenheim,Schalke 04,False,11.5,2.45,1.48,0.848326,0.077293,0.074381,58.898792
1,FRI3,2021-10-30,00:30:00,German Division 2 [GD2],Darmstadt,Nurnberg,False,10.5,2.20,1.59,0.822324,0.096388,0.081288,45.031567
2,SAT17,2021-10-30,19:30:00,German Division 2 [GD2],Dresden,Sandhausen,False,13.5,5.10,1.12,0.838060,0.086538,0.075402,44.426243
3,SAT18,2021-10-30,19:30:00,German Division 2 [GD2],Hannover,Aue,True,9.5,1.90,1.80,0.847085,0.096824,0.056091,46.659538
4,SAT19,2021-10-30,19:30:00,German Division 2 [GD2],Werder Bremen,St. Pauli,False,13.5,4.40,1.16,0.849211,0.090457,0.060333,54.170624
5,SAT93,2021-10-31,02:30:00,German Division 2 [GD2],Hamburg,Holstein Kiel,True,10.5,1.95,1.75,0.850449,0.085622,0.063929,61.205872
6,SUN12,2021-10-31,20:30:00,German Division 2 [GD2],Karlsruher,Paderborn,False,13.5,5.10,1.12,0.850045,0.088717,0.061238,65.328148
7,SUN13,2021-10-31,20:30:00,German Division 2 [GD2],Ingolstadt,Jahn Regensburg,True,9.5,2.00,1.72,0.846045,0.074061,0.079895,59.042469
8,SUN14,2021-10-31,20:30:00,German Division 2 [GD2],Rostock,Dusseldorf,True,9.5,1.75,1.95,0.843613,0.103807,0.052580,52.836399
9,FRI9,2021-10-30,02:30:00,German Division 1 [GSL],Hoffenheim,Hertha Berlin,True,9.5,1.80,1.90,0.826060,0.101464,0.072477,42.450790


In [42]:
prob_hilo = []

for r in list(zip(odds['prob_0'], odds['prob_1'], odds['prob_2'], odds['total_count'], odds['CHL_LINE'])):
    probs = torch.tensor(r[0:3], device='cpu')
    total_count = torch.tensor(r[3], device='cpu')

    bnb_corner = BivariateNegativeBinomial(total_count=total_count, probs=probs)
    value = torch.cartesian_prod(torch.arange(0., 15.), torch.arange(0., 15.))
    corner = bnb_corner.log_prob(value).exp()
    
    line = r[4]
    mask = value.sum(-1) < line
    prob_lo = corner[mask].sum()
    prob_hi = 1 - prob_lo
    
    prob_hilo.append([prob_hi.item(), prob_lo.item()])

In [43]:
odds[['prob_hi', 'prob_lo']] = prob_hilo

In [44]:
odds['kelly_hi'] = (odds['prob_hi'] * odds['CHL_H'] - 1) / (odds['CHL_H'] - 1)
odds['kelly_lo'] = (odds['prob_lo'] * odds['CHL_L'] - 1) / (odds['CHL_L'] - 1)

In [45]:
odds['kelly'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], odds['kelly_hi'], odds['kelly_lo']), 
    np.nan
)

In [46]:
odds['bet'] = np.where(
    np.maximum(odds['kelly_hi'], odds['kelly_lo']) > 0, 
    np.where(odds['kelly_hi'] > odds['kelly_lo'], 'High', 'Low'), 
    None
)

In [47]:
odds = odds.sort_values('kelly', ascending=False).reset_index(drop=True)

In [48]:
odds['selected'] = np.where(
    odds['MAINLINE']==True, np.where(
        odds['kelly']>0.3, '$$$', np.where(
            odds['kelly']>0.2, '$$', np.where(
                odds['kelly']>0.1, '$', None))), 
    None
)

In [49]:
odds = odds.drop(columns=cols_pred+['kelly_hi', 'kelly_lo'])

In [50]:
odds[odds.bet.notna() & odds.selected.notna()]

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
3,SAT27,2021-10-30,21:30:00,German Division 1 [GSL],Dortmund,Cologne,True,9.5,1.85,1.85,0.316134,0.683866,0.311945,Low,$$$
4,SUN12,2021-10-31,20:30:00,German Division 2 [GD2],Karlsruher,Paderborn,True,9.5,1.80,1.90,0.693715,0.306285,0.310859,High,$$$
10,SAT63,2021-10-30,22:15:00,Spanish Division 1 [SFL],Sevilla,Osasuna,True,7.5,1.67,2.07,0.696675,0.303325,0.243951,High,$$
11,SAT17,2021-10-30,19:30:00,German Division 2 [GD2],Dresden,Sandhausen,True,9.5,1.80,1.90,0.362267,0.637733,0.235214,Low,$$
12,SUN13,2021-10-31,20:30:00,German Division 2 [GD2],Ingolstadt,Jahn Regensburg,True,9.5,2.00,1.72,0.614781,0.385219,0.229562,High,$$
13,SAT18,2021-10-30,19:30:00,German Division 2 [GD2],Hannover,Aue,True,9.5,1.90,1.80,0.342677,0.657323,0.228977,Low,$$
22,SAT35,2021-10-30,22:00:00,Eng Premier [EPL],Watford,Southampton,True,10.5,2.02,1.70,0.344252,0.655748,0.163960,Low,$
23,SAT32,2021-10-30,22:00:00,Eng Premier [EPL],Manchester City,Crystal Palace,True,11.5,2.05,1.68,0.340078,0.659922,0.159806,Low,$
24,FRI9,2021-10-30,02:30:00,German Division 1 [GSL],Hoffenheim,Hertha Berlin,True,9.5,1.80,1.90,0.404852,0.595148,0.145313,Low,$
27,SUN38,2021-11-01,00:30:00,German Division 1 [GSL],Monchengladbach,Bochum,True,9.5,1.87,1.83,0.391446,0.608554,0.136932,Low,$


In [51]:
display_df(odds[odds.MatchDay.isin(odds[odds.bet.notna() & odds.selected.notna()].MatchDay) & odds.bet.notna()])

,MatchDay,Date,Time,LeagueJC,HomeTeamJC,AwayTeamJC,MAINLINE,CHL_LINE,CHL_H,CHL_L,prob_hi,prob_lo,kelly,bet,selected
0,SAT27,2021-10-30,21:30:00,German Division 1 [GSL],Dortmund,Cologne,False,13.5,4.90,1.13,0.055413,0.944587,0.518335,Low,None
1,SAT27,2021-10-30,21:30:00,German Division 1 [GSL],Dortmund,Cologne,False,10.5,2.30,1.54,0.219785,0.780215,0.373205,Low,None
2,SAT17,2021-10-30,19:30:00,German Division 2 [GD2],Dresden,Sandhausen,False,13.5,5.10,1.12,0.071448,0.928552,0.333151,Low,None
3,SAT27,2021-10-30,21:30:00,German Division 1 [GSL],Dortmund,Cologne,True,9.5,1.85,1.85,0.316134,0.683866,0.311945,Low,$$$
4,SUN12,2021-10-31,20:30:00,German Division 2 [GD2],Karlsruher,Paderborn,True,9.5,1.80,1.90,0.693715,0.306285,0.310859,High,$$$
5,SAT18,2021-10-30,19:30:00,German Division 2 [GD2],Hannover,Aue,False,13.5,5.55,1.10,0.063392,0.936608,0.302688,Low,None
6,SAT18,2021-10-30,19:30:00,German Division 2 [GD2],Hannover,Aue,False,10.5,2.35,1.52,0.241695,0.758305,0.293507,Low,None
7,SAT17,2021-10-30,19:30:00,German Division 2 [GD2],Dresden,Sandhausen,False,10.5,2.25,1.57,0.259206,0.740794,0.286046,Low,None
8,SUN38,2021-11-01,00:30:00,German Division 1 [GSL],Monchengladbach,Bochum,False,13.5,4.90,1.13,0.082454,0.917546,0.283288,Low,None
9,SUN12,2021-10-31,20:30:00,German Division 2 [GD2],Karlsruher,Paderborn,False,10.5,2.25,1.57,0.587290,0.412710,0.257122,High,None


In [52]:
odds.to_csv(path_output/f'odds-{datetime.now().strftime("%Y-%m-%d")}.csv', float_format='%.2f', index=False)

## END